
# Pipeline Deep Learning de génération d'images à partir de descriptions textuelles

Ce notebook présente, étape par étape, le code correspondant au script oral de 7 minutes. Chaque cellule de texte (Markdown) explique la partie évoquée dans le script, suivie de la cellule de code Python associée.  
La partie « Simulation avec modèle préentraîné » n’est **pas** incluse ici (vous avez déjà un programme pour cela).  



##  Objectif du projet

À partir d’un texte comme *"un chat blanc portant des lunettes"*, générer automatiquement une image correspondante.

Ce pipeline couvre :  
- La collecte des données  
- Le prétraitement  
- L'entraînement du modèle  
- Le déploiement dans une interface web  



##  Étape 1 : Choix du dataset

Nous avons utilisé **Flickr8k**, contenant 8 000 images avec 5 descriptions chacune.  
Avantages : léger, déjà aligné texte-image, bien documenté.

> Pour respecter les contraintes de Google Colab, nous avons sélectionné un **sous-ensemble de 2 000 paires** texte-image.


In [ ]:

# 1. Imports et Montage de Google Drive (pour sauvegarder les checkpoints)
import os
from google.colab import drive

# Montez votre Google Drive pour sauvegarder les checkpoints et éviter de perdre le travail
drive.mount('/content/drive')
save_dir = "/content/drive/MyDrive/text_to_image_project"
os.makedirs(save_dir, exist_ok=True)

# Librairies principales
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd


In [ ]:

# Charger les légendes et chemins d'images depuis un CSV (Flickr8k)
# (Assurez-vous d'avoir téléchargé les fichiers 'Flickr8k_text' sur Colab au préalable)
df = pd.read_csv("/content/Flickr8k_text/Flickr8k.token.txt",
                 sep='\t', names=['image', 'caption'])
# Sélection aléatoire d'un sous-ensemble de 2 000 paires
df = df.sample(n=2000, random_state=42).reset_index(drop=True)

# Affichage rapide pour vérifier
df.head()



##  Étape 2 : Prétraitement des données

**Pour les images :**  
- Redimension à 224×224 pixels  
- Conversion en tenseur PyTorch  
- Normalisation (valeurs entre 0 et 1)  

**Pour les textes :**  
- Tokenizer BERT (Hugging Face)  
- Conserver `input_ids` et `attention_mask`  

Nous créons une classe `FlickrDataset` qui, pour chaque exemple, renvoie :
- `image_tensor`  
- `input_ids`  
- `attention_mask`  


In [ ]:

# Import du tokenizer et des transformations PyTorch
from torchvision import transforms
from transformers import BertTokenizer

# Définition du tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Transformation pour les images
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),             # Convertit en [0,1]
])

class FlickrDataset(Dataset):
    def __init__(self, dataframe, image_dir, tokenizer, transform=None, max_length=32):
        self.data = dataframe
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_filename = row['image'].split('#')[0]  # Flickr8k.token.txt a format "IMG_####.jpg#0"
        img_path = os.path.join(image_dir, img_filename)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Tokenization du texte
        encoding = self.tokenizer(
            row['caption'],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )
        input_ids = encoding["input_ids"].squeeze(0)        # (max_length,)
        attention_mask = encoding["attention_mask"].squeeze(0)

        return image, input_ids, attention_mask

# Chemin vers le dossier contenant les images Flickr8k (ex: '/content/Flickr8k_Dataset/Images')
image_dir = "/content/Flickr8k_Dataset/Images"

# Instanciation du dataset et du DataLoader
dataset = FlickrDataset(df, image_dir, tokenizer, transform=image_transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)



##  Étape 3 : Construction du modèle

Nous définissons un modèle `DualBranchModel` qui contient :  
- **Branche Image :** ResNet50 préentraîné (dernière couche remplacée par une projection linéaire vers 256 dimensions).  
- **Branche Texte :** BERT préentraîné (on extrait `pooler_output`, puis projection en 256 dim).  

L’objectif : les embeddings texte et image d’une même paire doivent être proches.  


In [ ]:

import torch.nn as nn
from torchvision.models import resnet50
from transformers import BertModel

class DualBranchModel(nn.Module):
    def __init__(self, embed_dim=256):
        super().__init__()
        # Branche Image
        self.image_encoder = resnet50(pretrained=True)
        self.image_encoder.fc = nn.Linear(2048, embed_dim)

        # Branche Texte
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        self.text_proj = nn.Linear(768, embed_dim)  # 768 = taille d'embedding BERT

    def forward(self, images, input_ids, attention_mask):
        # Extraction embedding image
        image_embed = self.image_encoder(images)  # (batch_size, embed_dim)

        # Extraction embedding texte
        text_outputs = self.text_encoder(input_ids=input_ids,
                                         attention_mask=attention_mask)
        cls_embed = text_outputs.pooler_output       # (batch_size, 768)
        text_embed = self.text_proj(cls_embed)       # (batch_size, embed_dim)

        return image_embed, text_embed

# Instancier le modèle
model = DualBranchModel(embed_dim=256).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))



##  Étape 4 : Entraînement du modèle — Limitations rencontrées

On entraîne le modèle avec une **loss contrastive** (`CosineEmbeddingLoss`) pour rapprocher les embeddings correspondants.  
On sauvegarde le modèle après chaque epoch pour reprendre en cas de déconnexion.  


In [ ]:

import torch.nn.functional as F

# Définition de la fonction de perte
loss_fn = nn.CosineEmbeddingLoss(margin=0.0)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 3  # Réduire si nécessaire
model.train()

for epoch in range(num_epochs):
    total_loss = 0.0
    for images, input_ids, attention_mask in dataloader:
        images = images.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()
        image_embed, text_embed = model(images, input_ids, attention_mask)

        # cibles = +1 pour chaque paire (on veut similarité élevée)
        targets = torch.ones(images.size(0), device=device)
        loss = loss_fn(image_embed, text_embed, targets)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} — Loss : {avg_loss:.4f}")

    # Sauvegarde du modèle après chaque epoch
    checkpoint_path = os.path.join(save_dir, f"dual_model_epoch{epoch+1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"→ Modèle sauvegardé dans : {checkpoint_path}")



##  Étape 6 : Déploiement dans une interface Web (Gradio)

On recharge le modèle entraîné (dernier checkpoint) et on crée une fonction d’inférence qui,
à partir d’une légende textuelle, renvoie l’image la plus proche dans l’espace.  


In [ ]:

import gradio as gr

# Recharger le modèle entraîné (dernier checkpoint)
model = DualBranchModel(embed_dim=256).to(device)
last_checkpoint = os.path.join(save_dir, "dual_model_epoch3.pth")
model.load_state_dict(torch.load(last_checkpoint))
model.eval()

# Pré-calcul des embeddings images pour le sous-ensemble
image_embeddings = []
image_paths = []

for idx in range(len(dataset)):
    img, input_ids, attention_mask = dataset[idx]
    img = img.unsqueeze(0).to(device)
    with torch.no_grad():
        emb, _ = model(img,
                       input_ids.unsqueeze(0).to(device),
                       attention_mask.unsqueeze(0).to(device))
    image_embeddings.append(emb.cpu())
    img_filename = df.iloc[idx]['image'].split('#')[0]
    image_paths.append(os.path.join(image_dir, img_filename))

image_embeddings = torch.cat(image_embeddings, dim=0)  # (2000, 256)

def retrieve_image_from_caption(caption_text):
    encoding = tokenizer(
        caption_text, return_tensors="pt",
        padding="max_length", truncation=True, max_length=32
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        _, text_embed = model(None, input_ids, attention_mask)
    text_embed = text_embed.cpu()  # (1, 256)

    sims = torch.nn.functional.cosine_similarity(text_embed, image_embeddings)
    best_idx = torch.argmax(sims).item()
    best_image_path = image_paths[best_idx]

    return Image.open(best_image_path).convert("RGB")

iface = gr.Interface(
    fn=retrieve_image_from_caption,
    inputs=gr.inputs.Textbox(lines=2, placeholder="Entrez une description…"),
    outputs="image",
    title="Récupération d'image à partir d'une légende"
)

iface.launch(share=True)



## Conclusion

1. **Collecte de données** : Flickr8k, sous-ensemble 2 000 paires  
2. **Prétraitement** : redimension, normalisation, tokenisation BERT, extraction d’embeddings  
3. **Modèle dual** : ResNet + BERT + projection linéaire  
4. **Entraînement** : perte de similarité, sauvegarde des poids à chaque epoch  
5. **Simulation** : dans un autre notebook avec Stable Diffusion  
6. **Déploiement** : interface Gradio pour récupérer l’image la plus proche d’une légende  

Ce notebook fonctionne “normalement” (sans la partie simulation) et permet de relier chaque étape à votre script de présentation.  
